In [127]:
import numpy as np
import pandas as pd


In [2]:
def language_count(garbage):
    top_tier_languages = {'Python','C', 'C++', 'JavaScript', 'TypeScript', 'Ruby', 'Rust', 'Lua', 'Assembly', 'Zig'}
    return len(set(garbage.replace(' ', '').replace('{', '').replace('}','').replace("\'", '').split(',')) & top_tier_languages)


- 5 columns
- followesrs(index- 4)
- total_stars(index -6)
- total_forks(index -7)
- repo_count(inddex - 11)
- kino_languages_content (generate)

In [3]:
labeled_data = pd.read_csv('resources/labeled_data.csv')
labeled_data = np.array((labeled_data))
np.random.shuffle(labeled_data)


In [4]:
labeled_data[3]

array(['jackboespflug', 'https://api.github.com/users/jackboespflug',
       'https://github.com/jackboespflug',
       'https://api.github.com/users/jackboespflug/repos', 5,
       "[{'language': 'Shell', 'stars': 1, 'forks': 1, 'watchers': 1}, {'language': None, 'stars': 0, 'forks': 0, 'watchers': 0}, {'language': None, 'stars': 0, 'forks': 0, 'watchers': 0}, {'language': 'Elixir', 'stars': 1, 'forks': 1, 'watchers': 1}, {'language': None, 'stars': 0, 'forks': 0, 'watchers': 0}]",
       2, 2, 2, "{'Shell', 'Elixir'}", 1, 5], dtype=object)

In [128]:
kino_data = [data for data in labeled_data[:] if data[10] == 0]
slop_data = [data for data in labeled_data[:] if data[10] == 1][:5000]
synth_data = kino_data + slop_data
synth_data = np.array(synth_data)
synth_data.shape

(10184, 12)

In [50]:
np.random.shuffle(synth_data)

In [51]:
data = np.zeros((10000, 5), dtype=int)
data[:, :4] = synth_data[:10000, (4,6,7,11)].astype(int)
data[:, -1] = np.array([language_count(langs) for langs in labeled_data[:10000, 9]])
labels = synth_data[:10000, 10].astype(int)
one_hot_labels = np.zeros((len(labels), 2), dtype=int)
one_hot_labels[np.arange(len(labels)), labels] = 1
data.shape

(10000, 5)

In [53]:
len([i for i in labels if i==0])

5093

In [63]:
X_train, X_test = data[:8000], data[8000:]
y_train_true, y_test_true = labels[:8000], labels[8000:]
y_train, y_test =one_hot_labels[:8000],one_hot_labels[8000:]
X_train.shape, y_train.shape

((8000, 5), (8000, 2))

In [92]:

class Layer_Dense:
  def __init__(self,n_inputs,n_neurons):
    self.weights = 0.01 * np.random.randn(n_inputs,n_neurons)
    self.biases = np.zeros((1,n_neurons))

  def forward(self,inputs):
    self.output = np.dot(inputs, self.weights)+ self.biases

  def parameters(self):
    return self.weights


class Activation_ReLU:
  def forward(self,inputs):
    self.output = np.maximum(0,inputs)

class Activation_Softmax:
  def forward(self,inputs):
    exp_values = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
    probabilities = exp_values/np.sum(exp_values,axis=1,keepdims=True)
    self.output = probabilities


class Loss_Categoricalcrossentropy():
  def forward(self, probs, y_true):
    samples = len(probs)
    probs_clipped = np.clip(probs, 1e-7, 1 - 1e-7)
    #negative_log_likelihoods = -np.sum(y_true*np.log(probs_clipped))
    #final_loss = np.sum(negative_log_likelihoods)/samples
    correct_confidences = np.sum(probs_clipped * y_true, axis=1)
    n_l_l = -np.log(correct_confidences)
    final_loss = np.mean(n_l_l)
    return final_loss


In [119]:
layer1 = Layer_Dense(5,64)
layer2 = Layer_Dense(64,64)
layer3 = Layer_Dense(64,2)
activation1 = Activation_ReLU()
activation2 = Activation_ReLU()
activation3 = Activation_Softmax()

In [124]:
layer1.forward(X_test)
activation1.forward(layer1.output)
a1 = activation1.output
print(a1.shape)
layer2.forward(activation1.output)
activation2.forward(layer2.output)
a2 = activation2.output
print(a2.shape)
layer3.forward(activation2.output)
activation3.forward(layer3.output)
logits = activation3.output
print(logits.shape)

(2000, 64)
(2000, 64)
(2000, 2)


In [125]:
loss_function = Loss_Categoricalcrossentropy()
loss = loss_function.forward(logits,y_test)
print(f"Loss : {loss}")

y_pred = np.argmax(logits, axis=1)
accuracy = np.mean(y_pred == y_test_true)
print(f"Accuracy: {accuracy*100}")

Loss : 0.019153650529409323
Accuracy: 99.4


In [117]:
epochs = 100
batch_size = 20
learning_rate = 0.001

In [121]:
losses = []
accuracies = []

for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train_true[i:i+batch_size]
        y_true_batch = y_train[i:i+batch_size]

        layer1.forward(X_batch)
        activation1.forward(layer1.output)
        a1 = activation1.output

        layer2.forward(activation1.output)
        activation2.forward(layer2.output)
        a2 = activation2.output

        layer3.forward(activation2.output)
        activation3.forward(layer3.output)
        logits = activation3.output
        loss = loss_function.forward(logits,y_true_batch)
        epoch_loss += loss


        y_pred = np.argmax(activation3.output,axis=1)
        accuracy = np.mean(y_pred == y_batch)
        epoch_accuracy += accuracy

        d_loss = activation3.output - y_true_batch
        d_layer3 = d_loss
        d_layer2 = np.dot(d_layer3, layer3.weights.T) * (activation2.output > 0)
        d_layer1 = np.dot(d_layer2, layer2.weights.T) * (activation1.output > 0)

        # Update weights and biases
        layer3.weights -= learning_rate * np.dot(activation2.output.T, d_layer3)
        layer3.biases -= learning_rate * np.sum(d_layer3, axis=0, keepdims=True)
        layer2.weights -= learning_rate * np.dot(activation1.output.T, d_layer2)
        layer2.biases -= learning_rate * np.sum(d_layer2, axis=0, keepdims=True)
        layer1.weights -= learning_rate * np.dot(X_batch.T, d_layer1)
        layer1.biases -= learning_rate * np.sum(d_layer1, axis=0, keepdims=True)

    epoch_loss /= len(X_train) / batch_size
    epoch_accuracy /= len(X_train) / batch_size
    losses.append(epoch_loss)
    accuracies.append(epoch_accuracy)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")
    learning_rate*= 0.99

Epoch 1/100, Loss: 0.0173, Accuracy: 0.9944
Epoch 2/100, Loss: 0.0171, Accuracy: 0.9944
Epoch 3/100, Loss: 0.0170, Accuracy: 0.9944
Epoch 4/100, Loss: 0.0169, Accuracy: 0.9946
Epoch 5/100, Loss: 0.0169, Accuracy: 0.9945
Epoch 6/100, Loss: 0.0167, Accuracy: 0.9947
Epoch 7/100, Loss: 0.0166, Accuracy: 0.9949
Epoch 8/100, Loss: 0.0165, Accuracy: 0.9949
Epoch 9/100, Loss: 0.0164, Accuracy: 0.9946
Epoch 10/100, Loss: 0.0163, Accuracy: 0.9947
Epoch 11/100, Loss: 0.0162, Accuracy: 0.9950
Epoch 12/100, Loss: 0.0161, Accuracy: 0.9950
Epoch 13/100, Loss: 0.0160, Accuracy: 0.9949
Epoch 14/100, Loss: 0.0159, Accuracy: 0.9949
Epoch 15/100, Loss: 0.0158, Accuracy: 0.9950
Epoch 16/100, Loss: 0.0157, Accuracy: 0.9950
Epoch 17/100, Loss: 0.0157, Accuracy: 0.9951
Epoch 18/100, Loss: 0.0156, Accuracy: 0.9950
Epoch 19/100, Loss: 0.0155, Accuracy: 0.9952
Epoch 20/100, Loss: 0.0154, Accuracy: 0.9949
Epoch 21/100, Loss: 0.0154, Accuracy: 0.9950
Epoch 22/100, Loss: 0.0152, Accuracy: 0.9952
Epoch 23/100, Loss:

In [126]:
w1 = layer1.weights
w2 = layer2.weights
w3 = layer3.weights
b1 = layer1.biases
b2 = layer2.biases
b3 = layer3.biases
weights = {"w1" : w1, "w2" : w2, "w3" : w3, "b1" : b1, "b2" : b2, "b3" : b3}
np.save('resources/final_weights.npy', weights)